In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('data_processing')\
.getOrCreate()

23/11/02 14:37:53 WARN Utils: Your hostname, madgod resolves to a loopback address: 127.0.1.1; using 192.168.0.141 instead (on interface wlo1)
23/11/02 14:37:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 14:37:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/02 14:37:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/11/02 14:37:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.read.csv("./machine-learning-with-pyspark-master/chapter_2_Data_Processing/sample_data.csv",
                    inferSchema=True, header=True)

In [4]:
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [5]:
len(df.columns)

5

In [8]:
df.count()

33

In [9]:
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [10]:
df.show(3)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
+-------+---+----------+------+-------+
only showing top 3 rows



In [11]:
df.select('age','mobile').show(5)

+---+-------+
|age| mobile|
+---+-------+
| 32|   Vivo|
| 27|  Apple|
| 22|Samsung|
| 37|  Apple|
| 27|     MI|
+---+-------+
only showing top 5 rows



In [12]:
df.describe().show()

23/11/02 15:08:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  NULL|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  NULL|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



### Adding a New Column

In [13]:
df.withColumn("age_after_10_yrs", (df["age"]+10)).show(10, False)

+-------+---+----------+------+-------+----------------+
|ratings|age|experience|family|mobile |age_after_10_yrs|
+-------+---+----------+------+-------+----------------+
|3      |32 |9.0       |3     |Vivo   |42              |
|3      |27 |13.0      |3     |Apple  |37              |
|4      |22 |2.5       |0     |Samsung|32              |
|4      |37 |16.5      |4     |Apple  |47              |
|5      |27 |9.0       |1     |MI     |37              |
|4      |27 |9.0       |0     |Oppo   |37              |
|5      |37 |23.0      |5     |Vivo   |47              |
|5      |37 |23.0      |5     |Samsung|47              |
|3      |22 |2.5       |0     |Apple  |32              |
|3      |27 |6.0       |0     |MI     |37              |
+-------+---+----------+------+-------+----------------+
only showing top 10 rows



### Casting

In [14]:
from pyspark.sql.types import StringType, DoubleType

In [15]:
df.withColumn('age_double', df['age'].cast(DoubleType()))\
.show(10, False)

+-------+---+----------+------+-------+----------+
|ratings|age|experience|family|mobile |age_double|
+-------+---+----------+------+-------+----------+
|3      |32 |9.0       |3     |Vivo   |32.0      |
|3      |27 |13.0      |3     |Apple  |27.0      |
|4      |22 |2.5       |0     |Samsung|22.0      |
|4      |37 |16.5      |4     |Apple  |37.0      |
|5      |27 |9.0       |1     |MI     |27.0      |
|4      |27 |9.0       |0     |Oppo   |27.0      |
|5      |37 |23.0      |5     |Vivo   |37.0      |
|5      |37 |23.0      |5     |Samsung|37.0      |
|3      |22 |2.5       |0     |Apple  |22.0      |
|3      |27 |6.0       |0     |MI     |27.0      |
+-------+---+----------+------+-------+----------+
only showing top 10 rows



### Filtering Data

In [16]:
df.filter(df['mobile'] == 'Vivo').show()

+-------+---+----------+------+------+
|ratings|age|experience|family|mobile|
+-------+---+----------+------+------+
|      3| 32|       9.0|     3|  Vivo|
|      5| 37|      23.0|     5|  Vivo|
|      4| 37|       6.0|     0|  Vivo|
|      5| 37|      13.0|     1|  Vivo|
|      4| 37|       6.0|     0|  Vivo|
+-------+---+----------+------+------+



In [17]:
df.select('age','ratings','mobile').filter(df['mobile'] == 'Vivo').show()

+---+-------+------+
|age|ratings|mobile|
+---+-------+------+
| 32|      3|  Vivo|
| 37|      5|  Vivo|
| 37|      4|  Vivo|
| 37|      5|  Vivo|
| 37|      4|  Vivo|
+---+-------+------+



In [18]:
df.filter((df['mobile'] == 'Vivo') & (df['experience'] > 10)).show()

+-------+---+----------+------+------+
|ratings|age|experience|family|mobile|
+-------+---+----------+------+------+
|      5| 37|      23.0|     5|  Vivo|
|      5| 37|      13.0|     1|  Vivo|
+-------+---+----------+------+------+



### Distinct values in column

In [19]:
df.select('mobile').distinct().show()

+-------+
| mobile|
+-------+
|     MI|
|   Oppo|
|Samsung|
|   Vivo|
|  Apple|
+-------+



In [20]:
df.select('mobile').distinct().count()

5

### Grouping Data

In [21]:
df.groupBy('mobile').count().show(5, False)

+-------+-----+
|mobile |count|
+-------+-----+
|MI     |8    |
|Oppo   |7    |
|Samsung|6    |
|Vivo   |5    |
|Apple  |7    |
+-------+-----+



In [23]:
df.groupBy('mobile').count().orderBy('count', ascending=False)\
.show(5, False)

+-------+-----+
|mobile |count|
+-------+-----+
|MI     |8    |
|Oppo   |7    |
|Apple  |7    |
|Samsung|6    |
|Vivo   |5    |
+-------+-----+



In [24]:
df.groupBy('mobile').mean().show(5, False)

+-------+------------------+------------------+------------------+------------------+
|mobile |avg(ratings)      |avg(age)          |avg(experience)   |avg(family)       |
+-------+------------------+------------------+------------------+------------------+
|MI     |3.5               |30.125            |10.1875           |1.375             |
|Oppo   |2.857142857142857 |28.428571428571427|10.357142857142858|1.4285714285714286|
|Samsung|4.166666666666667 |28.666666666666668|8.666666666666666 |1.8333333333333333|
|Vivo   |4.2               |36.0              |11.4              |1.8               |
|Apple  |3.4285714285714284|30.571428571428573|11.0              |2.7142857142857144|
+-------+------------------+------------------+------------------+------------------+



In [25]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  NULL|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  NULL|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



In [26]:
df.groupBy('mobile').sum().show(5,False)

+-------+------------+--------+---------------+-----------+
|mobile |sum(ratings)|sum(age)|sum(experience)|sum(family)|
+-------+------------+--------+---------------+-----------+
|MI     |28          |241     |81.5           |11         |
|Oppo   |20          |199     |72.5           |10         |
|Samsung|25          |172     |52.0           |11         |
|Vivo   |21          |180     |57.0           |9          |
|Apple  |24          |214     |77.0           |19         |
+-------+------------+--------+---------------+-----------+



### Using Aggregate 

In [27]:
df.groupBy('mobile').agg({'experience':'sum'}).show()

+-------+---------------+
| mobile|sum(experience)|
+-------+---------------+
|     MI|           81.5|
|   Oppo|           72.5|
|Samsung|           52.0|
|   Vivo|           57.0|
|  Apple|           77.0|
+-------+---------------+



### User-Defined Functions(UDFs)

In [28]:
from pyspark.sql.functions import udf

#### Traditional Python Function

In [30]:
def price_range(brand):
    if brand in ['Samsung','Apple']:
        return 'High Price'
    elif brand == 'MI':
        return 'Mid Price'
    else:
        return 'Low Price'

In [31]:
brand_udf = udf(price_range,StringType())

In [32]:
df.withColumn('price_range',brand_udf(df['mobile'])).show()

+-------+---+----------+------+-------+-----------+
|ratings|age|experience|family| mobile|price_range|
+-------+---+----------+------+-------+-----------+
|      3| 32|       9.0|     3|   Vivo|  Low Price|
|      3| 27|      13.0|     3|  Apple| High Price|
|      4| 22|       2.5|     0|Samsung| High Price|
|      4| 37|      16.5|     4|  Apple| High Price|
|      5| 27|       9.0|     1|     MI|  Mid Price|
|      4| 27|       9.0|     0|   Oppo|  Low Price|
|      5| 37|      23.0|     5|   Vivo|  Low Price|
|      5| 37|      23.0|     5|Samsung| High Price|
|      3| 22|       2.5|     0|  Apple| High Price|
|      3| 27|       6.0|     0|     MI|  Mid Price|
|      2| 27|       6.0|     2|   Oppo|  Low Price|
|      5| 27|       6.0|     2|Samsung| High Price|
|      3| 37|      16.5|     5|  Apple| High Price|
|      5| 27|       6.0|     0|     MI|  Mid Price|
|      4| 22|       6.0|     1|   Oppo|  Low Price|
|      4| 37|       9.0|     2|Samsung| High Price|
|      4| 27

#### using Lambda Function

In [33]:
age_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())

In [34]:
df.withColumn("age_group", age_udf(df.age)).show()

+-------+---+----------+------+-------+---------+
|ratings|age|experience|family| mobile|age_group|
+-------+---+----------+------+-------+---------+
|      3| 32|       9.0|     3|   Vivo|   senior|
|      3| 27|      13.0|     3|  Apple|    young|
|      4| 22|       2.5|     0|Samsung|    young|
|      4| 37|      16.5|     4|  Apple|   senior|
|      5| 27|       9.0|     1|     MI|    young|
|      4| 27|       9.0|     0|   Oppo|    young|
|      5| 37|      23.0|     5|   Vivo|   senior|
|      5| 37|      23.0|     5|Samsung|   senior|
|      3| 22|       2.5|     0|  Apple|    young|
|      3| 27|       6.0|     0|     MI|    young|
|      2| 27|       6.0|     2|   Oppo|    young|
|      5| 27|       6.0|     2|Samsung|    young|
|      3| 37|      16.5|     5|  Apple|   senior|
|      5| 27|       6.0|     0|     MI|    young|
|      4| 22|       6.0|     1|   Oppo|    young|
|      4| 37|       9.0|     2|Samsung|   senior|
|      4| 27|       6.0|     1|  Apple|    young|


### Pandas UDF (Vectorized UDF)